<font size=8> Network Tests


In [4]:
#Common Python Packages
from pickle import TRUE
import pickle
import numpy as np
import matplotlib.pyplot as plt
import timeout_decorator

#dynapse2 spikegen imports
from lib.dynapse2_util import *
from lib.dynapse2_network import Network
from lib.dynapse2_spikegen import send_events,get_fpga_time, send_virtual_events, poisson_gen, isi_gen,regular_gen
from lib.dynapse2_raster import *
from lib.dynapse2_obj import *

#Statistics imports
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from itertools import product
#my imports
from adaptation import pc_single,ff,pc_pv_sst,ff_network
from adaptation_lib.spike_stats import *
from adaptation_lib.dynapse_setup import *
from adaptation_lib.graphing import *
from configs import neuron_configs_bio


# Determine the user's home directory
home_directory = os.path.expanduser("~")
# Path to the Documents directory
documents_path = os.path.join(home_directory, "Documents")
# Path to the dynapse-se2-data directory within Documents
base_path = os.path.join(documents_path, "dynapse-se2-data")
# Path to the random tests
random_search_path = os.path.join(documents_path, "network_random")

<font size=7> Manual Tuning


In [5]:
[board,profile_path,number_of_chips]=obtain_board()

In [6]:

def calculate_slope(time_axis, ff_windows_sst):
    slope, intercept, r_value, p_value, std_err = linregress(time_axis, ff_windows_sst)
    return slope


In [7]:
import random
neuron_config=neuron_configs_bio.config()

neuron_config['PC_Adaptation'] = True
neuron_config['duration'] = 1
neuron_config['input_type'] = 'Regular'
neuron_config['in_freq'] = 30

def randomize_config(neuron_config):
    neuron_config['SST_W0'] = [random.randint(1, 3), random.randint(10, 250)]
    neuron_config['SST_W1'] = [random.randint(1, 3), random.randint(10, 250)]
    neuron_config['SST_AMPA_GAIN'] = [random.randint(1, 3), random.randint(10, 250)]
    neuron_config['SST_AMPA_TAU'] = [random.randint(0, 1), random.randint(10, 250)]# 80-100
    neuron_config['SST_LEAK']= [1, random.randint(80, 100)] # 30-20
    neuron_config['Input_SST'] = random.uniform(.05, .5)
    neuron_config['PC_SST'] = random.uniform(.05, .5)

    return neuron_config


@timeout_decorator.timeout(40)  # Set a 5-second timeout

def run_simulation(neuron_config):
    return pc_pv_sst.pc_pv_sst(
        board=board,
        profile_path=profile_path,
        number_of_chips=number_of_chips,
        neuron_config=neuron_config
    )

successful_simulations = 0

for i in range(1000):  # Example: Keep trying different configurations until 100 successful simulations
    neuron_config = randomize_config(neuron_config)  # Assuming you've updated randomize_config without an argument
    try:
        [output_events, test_config] = run_simulation(neuron_config)
        [cv_values, synchrony_values] = run_dynamic_anal(output_events, test_config)
        fot_output= frequency_over_time(test_config, output_events)
        fvt_fig = frequency_vs_time_plot(frequency_over_time(test_config, output_events), test_config, save=True, annotate=True,neuron_config=neuron_config)
        [time_axis, ff_windows_pc, ff_windows_pv, ff_windows_sst] = fot_output
        slope = calculate_slope(time_axis, ff_windows_sst)
        print(f"Simulation {successful_simulations + 1} completed successfully!")
        successful_simulations += 1
        
        if successful_simulations >= 2000: # Stop after 100 successful simulations
            break
    except timeout_decorator.TimeoutError:
        print("Simulation timed out, trying again with new parameters...")

print("Completed 100 successful simulations.")


Setting Latches
Setting up network
Adding connections

Setting monitors
20 100 10

All configurations done!

Input events created
initilize run dynapse
dummy events sent

getting fpga time


setting virtual neurons

Simulation done
PC_CV_average: 0.04 PV_CV_average: 0.31 SST_CV_average: 0.0
PC_synchrony: 0.23 PV_CV_average: 0.48 SST_CV_average: nan
Simulation 1 completed successfully!
Setting Latches
Setting up network
Adding connections

Setting monitors
20 100 10

All configurations done!

Input events created
initilize run dynapse
dummy events sent

getting fpga time


setting virtual neurons

Simulation done
PC_CV_average: 0.04 PV_CV_average: 0.3 SST_CV_average: 0.0
PC_synchrony: 0.23 PV_CV_average: 0.51 SST_CV_average: nan
Simulation 2 completed successfully!
Setting Latches
Setting up network
Adding connections

Setting monitors
20 100 10

All configurations done!

Input events created
initilize run dynapse
dummy events sent

getting fpga time


setting virtual neurons

Simulatio

KeyboardInterrupt: 